# EECS 731 Final Project - Clothing One
## Classification

In [1]:
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
reviews = pickle.load(open("../Data/reviews.pkl", "rb" ))
rented_for_encoding = pickle.load(open("../Data/rented_for_encoding.pkl", "rb"))
body_type_encoding = pickle.load(open("../Data/body_type_encoding.pkl", "rb"))
category_encoding = pickle.load(open("../Data/category_encoding.pkl", "rb"))

In [3]:
reviews.head()

,age,body_type,category,fit,height,item_id,rating,rented_for,review_date,size,user_id,weight,bust_volume,bust_height
0,28,3,44,fit,68.0,2260466,10,6,2016-04-20,14,420272,137,590,4
1,36,6,20,fit,66.0,153475,10,3,2013-06-18,12,273551,132,390,2
2,34,4,16,fit,65.0,126335,8,2,2014-02-12,8,909926,135,480,3
3,27,1,20,fit,69.0,616682,10,7,2016-09-26,12,151944,145,390,2
4,45,1,16,fit,68.0,364092,8,0,2016-04-30,8,734848,138,310,2


In [4]:
# b = body_type_encoding.copy()
# b.reset_index(inplace=True)

# dictBody_type = b.to_dict()
# for i, row in b.iterrows():
#     reviews[row['labels']] = 0

# for index, row in reviews.iterrows():
#     reviews.at[index, dictBody_type['labels'][row['body_type']]] = 1

# reviews.drop(columns=['body_type'],inplace = True)
# reviews.head()

In [5]:
# b = rented_for_encoding.copy()
# b.reset_index(inplace=True)

# dictBody_type = b.to_dict()
# for i, row in b.iterrows():
#     reviews[row['labels']] = 0

# for index, row in reviews.iterrows():
#     reviews.at[index, dictBody_type['labels'][row['rented_for']]] = 1

# reviews.drop(columns=['rented_for'],inplace = True)
# reviews.head()

In [6]:
rented_for_encoding.head()

,encoding
labels,
date,0
everyday,1
formal affair,2
other,3
party,4


In [7]:
body_type_encoding.head()

,encoding
labels,
apple,0
athletic,1
full bust,2
hourglass,3
pear,4


In [8]:
category_encoding.head()

,encoding
labels,
ballgown,0
blazer,1
blouse,2
blouson,3
bomber,4


In [9]:
#Necessary imports
from sklearn.ensemble import RandomForestClassifier #Decision Tree Classifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split # can split data into test test & train set
from sklearn import metrics # Can calculate how accurrate our model is
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier

In [10]:
# removed user_id so we couldn't bank on history
# TODO: auto ml nn?
X = reviews.drop(['fit', 'rating', 'review_date'], axis=1)
Y = reviews['fit']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

# #Create Decision Tree and neural network classifer object
rf = RandomForestClassifier(n_estimators=100)
gb = GradientBoostingClassifier(random_state=1, learning_rate=.1)
nn = MLPClassifier()

# Train Decision Tree and neutral network Classifer
rf = rf.fit(X_train, Y_train)
gb = gb.fit(X_train, Y_train)
nn = nn.fit(X_train, Y_train)

#Predict the response for test dataset
rf_y_pred = rf.predict(X_test)
gb_y_pred = gb.predict(X_test)
nn_y_pred = nn.predict(X_test)


print("Random Forest Accuracy:",metrics.accuracy_score(Y_test, rf_y_pred))
print("Gradient Boosting Accuracy:",metrics.accuracy_score(Y_test, gb_y_pred))
print("Neural Network Accuracy:",metrics.accuracy_score(Y_test, nn_y_pred))

Random Forest Accuracy: 0.7319397479249923
Gradient Boosting Accuracy: 0.7357652764969088
Neural Network Accuracy: 0.7355261809611641


What % of users rate the fit of the product as "fit" to see if either model gave us additional information

In [11]:
print( (reviews[reviews.fit == 'fit'].shape[0]) / (len(reviews.index))  )

0.7359766636380405


More information is gained simply by guessing "fit" on every item, so these models provide us no additional information.

In [12]:
import numpy as np
importances = pd.DataFrame(gb.feature_importances_, index = X_train.columns, columns =['importance']).sort_values('importance', ascending=False)
importances

,importance
size,0.321014
item_id,0.218016
weight,0.205683
category,0.125788
rented_for,0.047647
age,0.022415
bust_volume,0.018113
body_type,0.017964
height,0.011069
user_id,0.009415


In [13]:
from sklearn.utils import resample
def resample_data(df):
    # Separate majority and minority classes
    df_majority = df[df.fit=='fit']
    df_minority_1 = df[df.fit=='small']
    df_minority_2 = df[df.fit=='large']

    size = df_majority.shape[0]
    
    # Upsample minority class
    df_minority_1_upsampled = resample(df_minority_1, 
                                     replace=True,     # sample with replacement
                                     n_samples=size,    # to match majority class
                                     random_state=123) # reproducible results
    
    df_minority_2_upsampled = resample(df_minority_2, 
                                     replace=True,     # sample with replacement
                                     n_samples=size,    # to match majority class
                                     random_state=123) # reproducible results
    # # Combine majority class with upsampled minority class
    df_upsampled = pd.concat([df_majority, df_minority_1_upsampled, df_minority_2_upsampled]).sample(frac=1)

    return df_upsampled

In [14]:
X_train['fit'] = Y_train.values
X_test['fit'] = Y_test.values

In [15]:
X_train_resampled = resample_data(X_train)
X_test_resampled = resample_data(X_test)

In [16]:
Y_train_resampled = X_train_resampled['fit']
Y_test_resampled = X_test_resampled['fit']
X_train_resampled = X_train_resampled.drop(['fit'],axis = 1)
X_test_resampled = X_test_resampled.drop(['fit'], axis = 1)

In [17]:
def fit_to_int(x):
    if x == 'fit':
        return 0
    elif x == 'large':
        return 1
    return -1
Y_train_resampled = Y_train_resampled.map(fit_to_int)
Y_test_resampled = Y_test_resampled.map(fit_to_int)

In [18]:
#Create Decision Tree and neural network classifer object
rf = RandomForestClassifier(n_estimators=100)
gb = GradientBoostingClassifier()
nn = MLPClassifier()

# Train Decision Tree and neutral network Classifer
rf = rf.fit(X_train_resampled, Y_train_resampled)
gb = gb.fit(X_train_resampled, Y_train_resampled)
nn = nn.fit(X_train_resampled, Y_train_resampled)

#Predict the response for test dataset
rf_y_pred = rf.predict(X_test_resampled)
gb_y_pred = gb.predict(X_test_resampled)
nn_y_pred = nn.predict(X_test_resampled)


print("Random Forest Accuracy:",metrics.accuracy_score(Y_test_resampled, rf_y_pred))
print("Gradient Boosting Accuracy:",metrics.accuracy_score(Y_test_resampled, gb_y_pred))
print("Neural Network Accuracy:",metrics.accuracy_score(Y_test_resampled, nn_y_pred))

Random Forest Accuracy: 0.3839354818736262
Gradient Boosting Accuracy: 0.503715055261447
Neural Network Accuracy: 0.3333333333333333


On the resample: Even thirds, 33% small, 33% fit, 33% large
Gradient Boosting here is 50% more accurate than random guessing

In [29]:
#Y_test_resampled.head()
a = pd.DataFrame(Y_test_resampled)
print(a.head())

print( (len(a[a['fit'] == 0])) / (len(a.index))  )

       fit
72332    0
68487    0
13685    0
33374   -1
93578    0
0.3333333333333333


In [19]:
import numpy as np
importances = pd.DataFrame(gb.feature_importances_, index = X_train_resampled.columns, columns =['importance']).sort_values('importance', ascending=False)
importances

,importance
size,0.418674
weight,0.195246
item_id,0.179305
category,0.084627
rented_for,0.057233
body_type,0.019177
bust_volume,0.018555
age,0.012474
height,0.006651
user_id,0.006181


In [20]:
Y_test = Y_test.map(fit_to_int)
X_train = X_train.drop(['fit'],axis = 1)
X_test = X_test.drop(['fit'],axis = 1)

In [21]:
gb_y_pred = gb.predict(X_test)
print("Gradient Boosting Accuracy:",metrics.accuracy_score(Y_test, gb_y_pred))

Gradient Boosting Accuracy: 0.3911944529835707


In [22]:
pd.DataFrame(Y_train_resampled).head(10)

,fit
131157,0
42631,1
57394,-1
70405,1
71454,1
79321,1
65162,-1
34424,1
135264,-1
64617,0
